## Word2Vec 문서유사도 prototype 구현해보기


In [2]:
# Library Import
import re
from tqdm import tqdm
import pandas as pd
import numpy as np

# Tokenize & Vectorize
from konlpy.tag import Okt
import gensim
from gensim.models import Word2Vec

# cosine similarity
from numpy import dot
from numpy.linalg import norm


In [4]:
# (참고) 데이터와 분석방향에 대한 간단한 설명

# 데이터: 약 2000여개의 책 데이터
# 분석방향: 책의 내용('context')을 분석해 책 A와 책 B가 얼마나 유사한지 "코사인 유사도"로 나타내고자 함

# ex. A라는 책과 B라는 책은 얼마나 유사할까?
# 책의 내용이 담긴 'context' 컬럼을 분석해 어떤 책끼리 유사한지 확인해보는 것이 목표이다
# [1. 전처리 2. 형태소 분석 3. word2vec모델에서 훈련 4. 코사인 유사도 분석] 순으로 진행한다


### 1. 데이터 전처리


In [5]:
# 데이터 호출
df = pd.read_csv('book.csv', encoding='cp949')


FileNotFoundError: [Errno 2] No such file or directory: 'book.csv'

In [ ]:
# 필요한 열만 뽑기
df_new = df_merge[['title', 'author', 'company', 'year', 'genre', 'context']]


In [ ]:
# 특수문자 처리
df_new['context'] = df_new['context'].str.replace(pat=r'[-=,/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》■□●○◆①②③④【】▶]', repl=r' ', regex=True)


In [ ]:
# 줄바꿈 문자 처리
df_new['context'] = df_new['context'].str.replace("\n", "")


In [ ]:
2. 형태소 분석하기


In [ ]:
# 형태소 분석기 twitter 사용을 위한 경로추가
import sys
sys.path.insert(0, '../')
import ckonlpy
print('ckonlpy version = {}'.format(ckonlpy.__version__))


In [ ]:
# 사용자 정의 사전 추가 : twitter가 인식하지 못하는 단어들을 새롭게 정의하기
twitter = Twitter(use_twitter_dictionary=False)
twitter.add_dictionary(['머신러닝','인공지능', '자율주행'])


In [ ]:
# 불용어 정의
# 형태소 분석에서 제외하고 싶은 단어를 정의
stopwords = ['하지만', '그래서', '특히']


In [ ]:
# 문장을 형태소 단위로 토큰화하는 함수 생성
def tokenizing(concat_str):
    
    temp_X = twitter.pos(concat_str, stem=True, norm=True)          # 단어 토큰화: 품사도 함께 표시되도록

    temp_X = [word[0] for word in temp_X if word[1] in ["Noun"]]    # 명사만 저장
    temp_X = [word for word in temp_X if not word in stopwords]     # 불용어 제거
    temp_X = [word for word in temp_X if len(word) > 1]             # 한글자짜리 형태소는 제거
    
    return temp_X


In [ ]:
# 'context' 컬럼에 토큰화 함수 적용 후, 결과를 'context_tokenized'이라는 새로운 컬럼에 넣는다.
df_new['context_tokenized'] = df_new['context'].apply(lambda x: tokenizing(x))


In [ ]:
# 토큰화 된 데이터를 tok_result 리스트에 넣는다.
tok_result = []
append = tok_result.append

for i in range(len(df_new['context_tokenized'])):
    append(df_new['ALLcontext_tokenizedtokenize'].values[i])


In [ ]:
3. Word2Vec 모델 훈련시키기


In [ ]:
# 모델 호출
# https://ratsgo.github.io/embedding/downloaddata.html 에서 다운로드한 word2vec 모델을 사용
# (4. 단어 임베딩 > '이곳' 클릭해 zip 다운로드 > word2vec 폴더 속 'word2vec'을 사용함)
model = gensim.models.Word2Vec.load('word2vec')


In [ ]:
# 위의 모델에 우리가 토큰화한 단어를 훈련시킨다.
model = Word2Vec(tok_result, size=100, window=5, min_count=5, workers=4, sg=0)


In [ ]:
# 개수 확인
total_examples = model.corpus_count
print(total_examples)


In [ ]:
# 모델의 추가훈련
# 위의 모델에는 없는 단어를 훈련시켜 성능을 향상시키기 위해 추가훈련을 진행한다.
# get_wiki_corpus.ipynb 에서 만든 모델을 불러온다 (약 53만건의 데이터를 포함하고 있는 모델)
new_model = gensim.models.Word2Vec.load('wiki_word2vec')


In [ ]:
# 새로운 모델에서 단어사전을 생성해 기존 모델에 붙이기
model.build_vocab([list(new_model.wv.vocab.keys())], update=True)


In [ ]:
# 모델 합치기
model.intersect_word2vec_format("ko.bin", binary=False)


In [ ]:
# 트레이닝
model.train(result, total_examples=total_examples, epochs=10)


In [ ]:
4. 단어 임베딩


In [ ]:
# model로부터 단어벡터 구하기
word_vectors = model.wv


In [ ]:
vocabs = list(word_vectors.vocab.keys())              # 단어
word_vectors_list = [word_vectors[v] for v in vocabs] # 벡터값


In [ ]:
# 딕셔너리 형태로 만들기: {'단어' : 벡터값}
vocab_vector_dict = dict(zip(vocabs, word_vectors_list))


In [ ]:
# 단어 벡터의 평균 구하기
def get_avg(context_tokenized):
    base_vector = np.zeros(shape = (100,))
    cnt = 0
    for token in context_tokenized:
        try:
            base_vector += vocab_vector_dict[token]
            cnt += 1
        except KeyError:
            continue
    return base_vector/cnt


In [ ]:
# 함수 적용 : 'context_vector' 컬럼에 벡터값을 저장한다
df_new['context_vector'] = df_new['context_tokenized'].apply(lambda x: get_avg(x))


In [ ]:
5. 코사인 유사도 계산


In [ ]:
# 코사인 유사도 구하는 함수 생성
def cos_sim(A, B):
    return dot(A, B) / (norm(A)*norm(B))


In [ ]:
# 최종적으로 생성하고자 하는 데이터프레임
final_df = pd.DataFrame(columns=['---1', '---2', '---3', '---4', '유사도'])


In [ ]:
for n in tqdm(range(len(df_new))):
    # 비교하고자 하는 데이터의 벡터값
    all_query_vector = df_new['context_vector'].values[n]

    # 같은 genre에 속하는 데이터끼리는 유사도를 계산하지 않는다.
    all_cos_list = []
    append = all_cos_list.append
    for context_vector in (df_new['context_vector'][df_new['genre'] != df_new.iloc[n]['genre']]).values:
        append(cos_sim(all_query_vector, context_vector))

    # 결과 DataFrame 생성
    result_df = pd.DataFrame(columns=['title', 'author', 'company', 'year', 'genre', '유사도'])

    # 결과로 도출된 데이터
    result_df['title'] = (df_new['title'][df_new['genre'] != df_new.iloc[n]['genre']]).values
    result_df['author'] = (df_new['author'][df_new['genre'] != df_new.iloc[n]['genre']]).values
    result_df['company'] = (df_new['company'][df_new['genre'] != df_new.iloc[n]['genre']]).values
    result_df['year'] = (df_new['year'][df_new['genre'] != df_new.iloc[n]['genre']]).values
    result_df['유사도'] = all_cos_list

    # 유사도 값 채우기
    result_df = result_df[np.isfinite(result_df['유사도'])]
    result_df = result_df.sort_values(by=['유사도'], ascending=False).reset_index(drop=True)

    # 비교하고자 하는 데이터
    result_df['제목'] = df_new['제목'].values[n]
    result_df['작가'] = df_new['작가'].values[n]
    result_df['출판사'] = df_new['출판사'].values[n]
    result_df['출판년도'] = df_new['출판년도'].values[n]

    # 유사도 0.90 이상의 데이터만 데이터프레임으로 저장한다.
    top_df = result_df.loc[result_df['유사도'] >= 0.90]

    # 생성된 데이터프레임을 합친다.
    final_df = pd.concat([final_df, top_df])


In [ ]:
# 유사도 높은 순으로 정렬
final_df.sort_values(['유사도'], ascending=False, inplace=True)


In [ ]:
# 기존 인덱스 번호로 되어있는 것을 리셋
final_df.reset_index(drop=True, inplace=True)


In [ ]:
# 생성된 데이터의 크기 확인
len(final_df)